In [2]:
import os 
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen =True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [5]:
from src.chickenDiseaseClassifier.constants import *# Step - creating entity
from src.chickenDiseaseClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config = CONFIG_FILE_PATH, param = PARAM_FILE_PATH):
        self.config = read_yaml(config)
        self.param = read_yaml(param)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        # This is a function from the os.path module that returns the directory name component of a given file path. 
        # It takes the file path as an argument and extracts the directory part, excluding the file name
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(root_dir = config.root_dir, 
                                                         tensorboard_root_log_dir = config.tensorboard_root_log_dir,
                                                         checkpoint_model_filepath = config.checkpoint_model_filepath
                                                        )
        return prepare_callback_config

In [8]:
import time
import tensorflow as tf


class PrepareCallback:
    def __init__(self,config : PrepareCallbacksConfig):
        self.config  =config
    
    @property
    #  This is a Python decorator that marks the method as a property.
    #  It allows the method to be accessed like an attribute without the need for explicit function call parentheses.
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tensorboard_logs_at_{timestamp}")
        

        return tf.keras.callbacks.TensorBoard(log_dir  = tb_running_log_dir)
        # The code snippet tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir) creates a TensorBoard callback 
        # instance in TensorFlow Keras. 
        # This callback is used to log training and evaluation metrics during the model training process at the specified position. 

    @property
    def create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(self.config.checkpoint_model_filepath,save_best_only=True)
    # It is used to save the model's weights during the training process

    def get_ckpt_tb_callbacks(self):
        return [self.create_tb_callbacks , self.create_checkpoint_callbacks]    

In [9]:
try:
    configuration_manager = ConfigurationManager()
    prepare_callbacks_config = configuration_manager.get_prepare_callback_config()
    prepare_callback = PrepareCallback(prepare_callbacks_config)
    callback_list = prepare_callback.get_ckpt_tb_callbacks()
except Exception as e:
    raise e

    # Note : seprate pipeline is not needed for this callbacks part . whenever we will do training at that time we will use
    # this callbacks methods 

[2023-08-07 12:59:03,246: INFO: common: yaml file : config\config.yaml loaded sucesfully]
[2023-08-07 12:59:03,261: INFO: common: yaml file : params.yaml loaded sucesfully]
[2023-08-07 12:59:03,268: INFO: common: Create directory at : artifacts]
[2023-08-07 12:59:03,274: INFO: common: Create directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2023-08-07 12:59:03,279: INFO: common: Create directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
